## Pandas dataframes

Pandas dataframes are a very convenient way to interact with low-dimensional structured data. The basic dataframe object acts very similarly to an Excel file, but data can be manipulated with Python rather than clumsy Excel functions.

In [6]:
%matplotlib inline
import pandas as pd

#Look at csv file 
df = pd.read_csv('/Users/asus/F19-4803DR-8843DR/oceanic_data-brewer.csv',skiprows=32)
#df['DATE']
#df

Pandas dataframes are a cross between dictionaries and numpy arrays. Unlike arrays, they are allowed to hold multiple types, and they index columns and rows based on "keys" rather than numbers:

In [7]:
#print(df.index) #row names
print(df.columns) # column names
#print(type(df.values)) #all data
#print(df.dtypes)

Index(['EXPOCODE', 'STNNBR', 'CASTNO', 'BTLNBR', 'BTLNBR_FLAG_W', 'DATE',
       'TIME', 'LATITUDE', 'LONGITUDE', 'DEPTH', 'CTDPRS', 'CTDTMP',
       'CTDTMP_FLAG_W', 'SALNTY', 'SALNTY_FLAG_W', 'OXYGEN', 'OXYGEN_FLAG_W',
       'SILCAT', 'SILCAT_FLAG_W', 'NITRAT', 'NITRAT_FLAG_W', 'NITRIT',
       'NITRIT_FLAG_W', 'PHSPHT', 'PHSPHT_FLAG_W', 'TCARBN', 'TCARBN_FLAG_W',
       'ALKALI', 'ALKALI_FLAG_W', 'TRITUM', 'TRITER', 'HELIUM', 'DELHE3',
       'DELC14', 'C14ERR', 'DELC14_FLAG_W', 'RA-226', 'RA-226E',
       'RA-226_FLAG_W', 'RA-228', 'RA-228E', 'RA-228_FLAG_W', 'RA-8/6',
       'RA-8/6E', 'RA-8/6_FLAG_W', 'THETA'],
      dtype='object')


It is easy to view and summarize data with the dataframe object:

In [11]:
df.head(5)
# df.tail(10)
#df

,EXPOCODE,STNNBR,CASTNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,LONGITUDE,DEPTH,...,RA-226,RA-226E,RA-226_FLAG_W,RA-228,RA-228E,RA-228_FLAG_W,RA-8/6,RA-8/6E,RA-8/6_FLAG_W,THETA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METERS,...,DPM/100KG,DPM/100KG,NaN,DPM/100KG,DPM/100KG,NaN,NaN,NaN,NaN,DEG C
1,TTONAS,1.0,2.0,27.0,2.0,19810402.0,0.0,39.805,-70.082,1172,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,11.2656
2,TTONAS,1.0,2.0,28.0,2.0,19810402.0,0.0,39.805,-70.082,1172,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,11.2142
3,TTONAS,1.0,2.0,29.0,2.0,19810402.0,0.0,39.805,-70.082,1172,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,12.6338
4,TTONAS,1.0,2.0,30.0,2.0,19810402.0,0.0,39.805,-70.082,1172,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,8.5224


Data can be accessed in several ways:

* Columns can be accessed directly with keys. 
* The `loc` method enables numpy-like indexing, fancy indexing, and slicing. 
* The `iloc` method is similar to `loc`, but indexes by position (rather than key)

These methods return `pandas.Series` objects, that are basically 1D dataframes.

In [10]:
#df['LATITUDE']
#df.loc[1,'LATITUDE']
df.loc[6]
#df.loc[:,['LATITUDE','LONGITUDE']]
#df.iloc[:,7]

EXPOCODE                 TTONAS
STNNBR                        1
CASTNO                        2
BTLNBR                       33
BTLNBR_FLAG_W                 2
DATE                1.98104e+07
TIME                          0
LATITUDE                 39.805
LONGITUDE               -70.082
DEPTH                      1172
CTDPRS                    813.0
CTDTMP                   4.5840
CTDTMP_FLAG_W                 2
SALNTY                  34.9630
SALNTY_FLAG_W                 2
OXYGEN                    250.0
OXYGEN_FLAG_W                 2
SILCAT                    12.50
SILCAT_FLAG_W                 2
NITRAT                    18.60
NITRAT_FLAG_W                 2
NITRIT                     0.00
NITRIT_FLAG_W                 2
PHSPHT                     1.22
PHSPHT_FLAG_W                 2
TCARBN                   2169.0
TCARBN_FLAG_W                 2
ALKALI                   2331.0
ALKALI_FLAG_W                 2
TRITUM                    2.470
TRITER                    0.060
HELIUM  

Columns can be deleted in three ways:

* del : delete the Series from the dataframe
* pop() : delete the Series and return the Series
* drop(labels, axis) : return a new dataframe with Series removed (do not modify original df)

Rows must be "dropped".

**Pay attention to whether operations are "in place" or not**. Many `pandas` operations are *not* "in place" by default. This means that they return a copy of the dataframe with modifications, rather than modifying the original dataframe object. This can be very confusing.

In [12]:
#Delete column
times = df.pop('TIME')
print(times)
del df['EXPOCODE']
dfnew = df.drop('CASTNO', axis=1)

#Delete row
df2 = df.drop(df.index[0])
df2 = df2.drop(df.index[-1])
df2.head(3)

0       NaN
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
9144    0.0
9145    0.0
9146    0.0
9147    0.0
9148    0.0
9149    0.0
9150    0.0
9151    0.0
9152    0.0
9153    0.0
9154    0.0
9155    0.0
9156    0.0
9157    0.0
9158    0.0
9159    0.0
9160    0.0
9161    0.0
9162    0.0
9163    0.0
9164    0.0
9165    0.0
9166    0.0
9167    0.0
9168    0.0
9169    0.0
9170    0.0
9171    0.0
9172    0.0
9173    NaN
Name: TIME, Length: 9174, dtype: float64


,STNNBR,CASTNO,BTLNBR,BTLNBR_FLAG_W,DATE,LATITUDE,LONGITUDE,DEPTH,CTDPRS,CTDTMP,...,RA-226,RA-226E,RA-226_FLAG_W,RA-228,RA-228E,RA-228_FLAG_W,RA-8/6,RA-8/6E,RA-8/6_FLAG_W,THETA
1,1.0,2.0,27.0,2.0,19810402.0,39.805,-70.082,1172,11.0,11.2670,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,11.2656
2,1.0,2.0,28.0,2.0,19810402.0,39.805,-70.082,1172,55.0,11.2210,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,11.2142
3,1.0,2.0,29.0,2.0,19810402.0,39.805,-70.082,1172,156.0,12.6550,...,-999.00,-999.00,-999.0,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,12.6338


Let's simplify this data into only the columns that we care about:

In [ ]:
interesting = ['DATE','LATITUDE','LONGITUDE','DEPTH','SALNTY','OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
df = df[interesting]
df = df.drop(df.index[0])
df = df.drop(df.index[-1])
df

It looks like there are some artifacts since a concentration of -999.0 does not make sense. We can get rid of these using "boolean indexing":

In [ ]:
df.dtypes
df = df.astype('float')
df.dtypes
conc = ['OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
for var in conc:
    df = df[df[var] >= 0] #only take rows where concentration is >=0

df = df.reset_index() #<- THIS IS IMPORTANT!!!
df

This could also be achieved with OpenRefine. There is often no right way to do things in data science, so pick whatever feels easiest or most intuitive.

Note that the dates were not parsed correctly. We can fix this with a loop and assign the proper dates to the DATE column:

In [ ]:
def convert_DATE(date_col):
    dates = []
    for dt in date_col:
        dt = str(int(dt))
        y,m,d = dt[:4], dt[4:6], dt[6:]
        dates.append('-'.join([y,m,d]))
    print(len(dates))
    dates = pd.Series(dates)
    dates = pd.to_datetime(dates,yearfirst=True)
    return dates

dates = convert_DATE(df['DATE'])
df['DATE'] = dates # [(0, date0), (1, date1) ... ]

In [ ]:
df.head(10)

Pandas dataframes have some handy plotting features built in:

In [ ]:
print(conc)
df[conc].plot()
df_sorted = df.sort_values('PHSPHT')
df_sorted.plot(x='PHSPHT',y='SILCAT')
df_sorted.hist('SILCAT')

Pandas allows easily indexing by different columns:

In [ ]:
df2 = df.set_index('DATE')
df2['1981-06-03':'1981-09-07']

## Pandas multi-indexing

Pandas multi-indexing allows multi-dimensional `DataFrame` objects that act like different sheets/files in Excel. This is very useful for creating "mini databases" that allow organization of complex data with multiple dimensions.

Let's consider another dataset similar to the one we just worked with:

In [ ]:
new_df = pd.read_csv('datasets/oceanic_data-morliere.csv',skiprows=35)
#quickly clean up the data:
new_df = new_df[interesting]
new_df = new_df.drop(0)
new_df = new_df.astype('float')
for var in conc:
    new_df = new_df[new_df[var] > 0]
new_df = new_df.reset_index()
dates = convert_DATE(new_df['DATE'])
new_df['DATE'] = dates
new_df = new_df.set_index('DATE')
new_df

Now we might want to create a DateFrame that contains both sets of data, but index them by the scientist that collected the data (Brewer and Morliere). This can be achieved with the Multi Index functionality:

In [ ]:
brewer = df2
morliere = new_df
full = pd.concat({'Brewer':brewer, 'Morliere':morliere})
#print(full.columns)
full = full.T
#print(full.columns)
x = full['Morliere'].T['1993-02'] #<- get all measurements by Brewer in May 1981
print(x.columns)
x.head(10)

It is easy to convert from a `pandas` dataframe back to a `numpy` array (assuming all values are numerical):

In [ ]:
x.values